In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
zip_ref = zipfile.ZipFile("drive/My Drive/My Fancy Projects/Skin Disease Diagnosis/dataset.zip", 'r')
zip_ref.extractall("/content/skindata/")
zip_ref.close()

In [ ]:
from numpy import zeros
from numpy import ones
from numpy import expand_dims
from numpy.random import randn
from numpy.random import randint
from keras.datasets.fashion_mnist import load_data
from keras.optimizers import Adam,SGD, RMSprop,Adadelta
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Add
from keras.layers import Flatten
from keras.initializers import RandomNormal
from matplotlib import pyplot
from tqdm import tqdm
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import time
import tensorflow as tf

In [ ]:
import keras
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
#from keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.densenet import DenseNet121
from keras.layers import GlobalMaxPooling2D, BatchNormalization
from keras.utils import plot_model

import os
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
from keras import layers
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping


pre_trained_model = tf.keras.applications.Xception(include_top=False,
                        weights='imagenet',
                        input_tensor=None,
                        input_shape=(150, 150, 3))


i = -16
layers = []



for layer in pre_trained_model.layers:
  layers.append(layer)

non_trainable_layers = layers[:i]
trainable_layers = layers[i:]


for layer in non_trainable_layers:
  layer.trainable = False


for layer in trainable_layers:
  layer.trainable = True


In [ ]:
pre_trained_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [ ]:
print(len(pre_trained_model.layers))

132


In [ ]:
model_1 = Model(inputs = pre_trained_model.input, 
                outputs =pre_trained_model.get_layer('add_9').output )

adder = Add()([model_1.output,pre_trained_model.get_layer('add_2').output])

model_2 = Model(inputs = pre_trained_model.input, 
               outputs = pre_trained_model.get_layer('add_10').output )

adder = Add()([model_2.output,pre_trained_model.get_layer('add_4').output])

model_3 = Model(inputs = adder,
                 outputs = pre_trained_model.output)

model = Model(inputs = model_1.input,
              outputs = model_3.output)

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: add_13/add:0


ValueError: ignored

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
from keras.layers.normalization import BatchNormalization
from keras.layers import ReLU

neck = GlobalMaxPooling2D()(pre_trained_model.output)

#neck = GlobalMaxPooling2D()(model.output)

In [ ]:
print(pre_trained_model.output.shape)

(None, 5, 5, 2048)


In [ ]:
"""
def mid_flow(in_layer):

    neck = Add()([pre_trained_model.get_layer('block3_pool').output,in_layer])

    return neck
"""

def build_tower(in_layer):
  	
    x = pre_trained_model.get_layer('block3_pool').output
    x = Flatten()(x)
    neck = Dense(18*18*256)(in_layer)
    neck = Add()([x,neck])
    neck = BatchNormalization()(neck)
    neck = Dropout(0.2)(neck)
    neck = ReLU()(neck)


    """
    x = pre_trained_model.get_layer('block6_sepconv3').output
    x = Flatten()(x)
    neck = Dense(9*9*728)(neck)
    neck = Add()([x,neck])
    neck = BatchNormalization()(neck)
    neck = Dropout(0.2)(neck)
    neck = ReLU()(neck)
  	"""
    
    neck = Dense(512)(neck)
    neck = BatchNormalization()(neck)
    neck = Dropout(0.2)(neck)
    neck = ReLU()(neck)

    neck = Dense(256)(neck)
    neck = BatchNormalization()(neck)
    neck = ReLU()(neck)

    return neck


def build_head(in_layer):
    return Dense(5, activation='softmax', name=f'Skin_Disease_Output')(in_layer)    

#action = build_head(build_tower(mid_flow(neck)))

action = build_head(build_tower(neck))


model = Model(
    inputs=pre_trained_model.input,
    outputs=[action],

    name = 'model_xception'
)

In [ ]:
model.summary()

Model: "model_xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
_____________________________________________________________________________________

In [ ]:
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-8,
    verbose=1,
    min_delta=1e-4
)
early_stopping = EarlyStopping(
    patience=10,
    verbose=1,
    min_delta=1e-4,
    restore_best_weights=False
)
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<0.0001):
            print("\nReached 0.0001 loss so cancelling training!")
            self.model.stop_training 


from keras.optimizers import Adam

def compile_model(model):
    model.compile(
        optimizer=Adam(lr=0.01),
        loss ='categorical_crossentropy',
        metrics=["acc"]
    )
compile_model(model)

In [ ]:
train_datapath = "/content/skindata/dataset/train/"
test_datapath = "/content/skindata/dataset/test"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        train_datapath,
        target_size=(150, 150),
        class_mode='categorical')


test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
        test_datapath,
        target_size=(150, 150),
        batch_size=20,
        class_mode='categorical')


Found 4363 images belonging to 5 classes.
Found 1194 images belonging to 5 classes.


In [ ]:
import datetime
from keras.callbacks import TensorBoard

In [ ]:
logpath = "drive/My Drive/My Fancy Projects/Skin Disease Diagnosis/logs/"
logdir = os.path.join(logpath, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
history = model.fit(train_generator,  
                    validation_data=validation_generator,                                                      
                    epochs = 100,
                    verbose = 1,
                    use_multiprocessing=False, 
                    callbacks = [myCallback(),tensorboard_callback,reduce_lr_on_plateau,early_stopping]) #early_stopping

Epoch 1/100
137/137 [==============================] - 66s 434ms/step - loss: 1.5532 - acc: 0.4310 - val_loss: 7.0800 - val_acc: 0.4447
Epoch 2/100
137/137 [==============================] - 57s 415ms/step - loss: 1.2278 - acc: 0.5220 - val_loss: 1.6610 - val_acc: 0.4439
Epoch 3/100
137/137 [==============================] - 56s 411ms/step - loss: 1.1787 - acc: 0.5410 - val_loss: 1.9484 - val_acc: 0.3987
Epoch 4/100
137/137 [==============================] - 57s 414ms/step - loss: 1.0972 - acc: 0.5738 - val_loss: 1.2377 - val_acc: 0.5519
Epoch 5/100
137/137 [==============================] - 56s 412ms/step - loss: 1.0130 - acc: 0.6061 - val_loss: 1.1561 - val_acc: 0.6106
Epoch 6/100
137/137 [==============================] - 56s 411ms/step - loss: 1.0156 - acc: 0.6068 - val_loss: 1.1406 - val_acc: 0.6156
Epoch 7/100
137/137 [==============================] - 56s 411ms/step - loss: 0.9347 - acc: 0.6442 - val_loss: 1.1762 - val_acc: 0.5913
Epoch 8/100
137/137 [===========================